### Imports
---

In [1]:
import pandas as pd

### Load data
---

In [2]:
dataframe = pd.read_csv("src/data/HI-Small_Trans.csv")

In [3]:
dataframe.head(2)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0


In [4]:
dataframe.shape

(5078345, 11)

### Transformations
---

In [5]:
filtered_dataframe = dataframe[dataframe["Amount Paid"] == dataframe["Amount Received"]]

In [6]:
filtered_dataframe.columns = filtered_dataframe.columns.str.lower()
filtered_dataframe.columns = filtered_dataframe.columns.str.replace(' ', "_")

In [7]:
filtered_dataframe[["from_bank", "to_bank"]] = filtered_dataframe[["from_bank", "to_bank"]].astype(str)

C:\Users\ferna\AppData\Local\Temp\ipykernel_8488\1763207121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe[["from_bank", "to_bank"]] = filtered_dataframe[["from_bank", "to_bank"]].astype(str)


In [8]:
filtered_dataframe.head(2)

,timestamp,from_bank,account,to_bank,account.1,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0


In [9]:
filtered_dataframe["sender"] = filtered_dataframe["from_bank"] + "_" + filtered_dataframe["account"]
filtered_dataframe["receiver"] = filtered_dataframe["to_bank"] + "_" + filtered_dataframe["account.1"]

C:\Users\ferna\AppData\Local\Temp\ipykernel_8488\3560939121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe["sender"] = filtered_dataframe["from_bank"] + "_" + filtered_dataframe["account"]
C:\Users\ferna\AppData\Local\Temp\ipykernel_8488\3560939121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe["receiver"] = filtered_dataframe["to_bank"] + "_" + filtered_dataframe["account.1"]


In [10]:
filtered_dataframe = filtered_dataframe[[
 "timestamp",
 "sender",
 "receiver",
 "amount_received",
 "receiving_currency",
 "amount_paid",
 "payment_currency",
 "payment_format",
 "is_laundering",
]]

In [11]:
filtered_dataframe = filtered_dataframe.query("sender != receiver")

In [12]:
filtered_dataframe.to_csv("src/data/machine_learning_data.csv", index=False)

In [13]:
filtered_dataframe = filtered_dataframe.groupby(["sender", "receiver"], as_index=False).agg({"amount_paid":  "sum"})
filtered_dataframe.columns = ['sender','receiver','sum']
filtered_dataframe = filtered_dataframe.rename(columns={"sum": "value"})
filtered_dataframe["title"] = filtered_dataframe.apply(lambda df: f"from: {df.sender}\nto: {df.receiver}\namount (BRL): {df.value}", axis=1)

In [15]:
filtered_dataframe.to_csv("src/data/network_analysis_data.csv", index=False)